# Imports

In [1]:
import random
import pandas as pd
import numpy as np
import tensorflow as tf

from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense, concatenate, Input, BatchNormalization
from tensorflow import keras
from sklearn.model_selection import train_test_split

# Connect drive

In [2]:
from google.colab import drive

drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


# Dataset

In [3]:
path_images="/content/drive/My Drive/uc project/images.npz"
dict_image_data = np.load(path_images)
image_data = dict_image_data['arr_0']

In [4]:
path_cities="/content/drive/My Drive/uc project/cities.npz"
dict_city_data = np.load(path_cities)
city_data = dict_city_data['arr_0']

path_class="/content/drive/My Drive/uc project/classes.npz"
dict_class_data = np.load(path_class)
class_data = dict_class_data['arr_0']

In [5]:
# all 512 images resized to 128
# data_512 = image_data[:,0][indices]
data_512 = image_data[:,0]
# all 512 images cropped to 256 images and resized to 128
# data_256 = image_data[:,1][indices]
data_256 = image_data[:,1]
# all 512 images cropped to 128 images
# data_128 = image_data[:,2][indices]
data_128 = image_data[:,2]

In [6]:
city_data.shape, data_512.shape, data_512.shape

((28195,), (28195, 128, 128, 3), (28195, 128, 128, 3))

Tranforming labels to onehot encoding

In [7]:
from sklearn.preprocessing import LabelBinarizer
label_as_binary = LabelBinarizer()

city_labels = label_as_binary.fit_transform(city_data)
class_labels = label_as_binary.fit_transform(class_data)

Split train and test data


In [8]:
train_512, test_512, train_256, test_256, train_128, test_128, train_city, test_city, train_class, test_class = train_test_split(data_512, data_256, data_128, city_labels, class_labels, test_size=0.2, random_state=42)

In [9]:
from collections import Counter

Counter(np.argmax(train_city, axis=1))

Counter({0: 4031, 1: 4340, 3: 6313, 2: 3678, 4: 4194})

In [10]:
Counter(np.argmax(test_city, axis=1))

Counter({2: 980, 1: 1126, 0: 991, 3: 1527, 4: 1015})

In [11]:
Counter(np.argmax(train_class, axis=1))

Counter({3: 1446,
         9: 909,
         0: 6836,
         4: 1520,
         13: 7428,
         11: 1185,
         10: 650,
         6: 608,
         17: 369,
         14: 117,
         16: 362,
         2: 99,
         15: 373,
         8: 255,
         5: 205,
         1: 132,
         7: 37,
         12: 25})

In [12]:
Counter(np.argmax(test_class, axis=1))

Counter({13: 1870,
         11: 338,
         4: 391,
         10: 171,
         9: 225,
         2: 28,
         0: 1681,
         6: 139,
         3: 332,
         14: 30,
         5: 55,
         17: 100,
         16: 89,
         15: 91,
         8: 54,
         1: 35,
         7: 7,
         12: 3})

# Networks

In [13]:
def basic_model(input_name):
  # 1
  model_input = Input(shape=(128, 128, 3), name=input_name)
  conv2d1 = Conv2D(filters=32, kernel_size=(3,3), activation='relu')(model_input)
  # norm1 = BatchNormalization()(conv2d1)
  # 2
  pool1 = MaxPooling2D(pool_size=(2,2))(conv2d1)
  # norm2 = BatchNormalization()(pool1)
  # 1
  conv2d2 = Conv2D(filters=32, kernel_size=(3,3), activation='relu')(pool1)
  # norm3 = BatchNormalization()(conv2d2)
  # 2
  pool2 = MaxPooling2D(pool_size=(2,2))(conv2d2)
  # norm4 = BatchNormalization()(pool2)
  # 15
  flat = Flatten()(pool2)
  model_output = Dense(2048, activation='relu')(flat)
  # model_output = Dropout(0.1)(dense)
  return model_input, model_output

## e128

Get input and initial output layers

In [14]:
basic_input, basic_output = basic_model('main_input')

Attach 2 outputs

In [15]:
#City
merged_hidden_city = Dense(2048, activation='relu')(basic_output)
# merged_drop_city = Dropout(0.1)(merged_hidden_city)
city_output = Dense(len(np.unique(city_data)), activation='softmax', name='city_output')(merged_hidden_city)

#Class
merged_hidden_class = Dense(2048, activation='relu')(basic_output)
# merged_drop_class = Dropout(0.1)(merged_hidden_class)
class_output = Dense(len(np.unique(class_data)), activation='softmax', name='class_output')(merged_hidden_class)

In [16]:
e128_model = Model(inputs=basic_input, outputs=[city_output, class_output])
# e128_model.compile(optimizer='adam', loss=['categorical_crossentropy', 'categorical_crossentropy'], metrics=['accuracy'])
gamma = 0.5
e128_model.compile(optimizer='adam',
                  loss={'city_output': 'categorical_crossentropy', 
                        'class_output': 'categorical_crossentropy'},
                  loss_weights={'city_output': gamma, 
                                'class_output': 1 - gamma}, 
                  metrics=['accuracy'])

In [17]:
e128_history = e128_model.fit(train_128, {'city_output': train_city, 'class_output': train_class},
                              # batch_size=20,
                              epochs=10,
                              shuffle=True,
                              validation_data=(test_128, {'city_output': test_city, 'class_output': test_class})
                 )

Epoch 1/5
1128/1128 [==============================] - 28s 23ms/step - loss: 6.9141 - city_output_loss: 7.3016 - class_output_loss: 6.5267 - city_output_accuracy: 0.3419 - class_output_accuracy: 0.3789
Epoch 2/5
1128/1128 [==============================] - 25s 22ms/step - loss: 1.5350 - city_output_loss: 1.3179 - class_output_loss: 1.7521 - city_output_accuracy: 0.4328 - class_output_accuracy: 0.4207
Epoch 3/5
1128/1128 [==============================] - 25s 22ms/step - loss: 1.2681 - city_output_loss: 1.1175 - class_output_loss: 1.4186 - city_output_accuracy: 0.5364 - class_output_accuracy: 0.5258
Epoch 4/5
1128/1128 [==============================] - 25s 22ms/step - loss: 0.9246 - city_output_loss: 0.8982 - class_output_loss: 0.9509 - city_output_accuracy: 0.6367 - class_output_accuracy: 0.6807
Epoch 5/5
1128/1128 [==============================] - 25s 23ms/step - loss: 0.7084 - city_output_loss: 0.7087 - class_output_loss: 0.7081 - city_output_accuracy: 0.7136 - class_output_accurac

## e256

Get input and initial output layers

In [18]:
basic_input, basic_output = basic_model('main_input')

Attach 2 outputs

In [19]:
#City
merged_hidden_city = Dense(2048, activation='relu')(basic_output)
# merged_drop_city = Dropout(0.1)(merged_hidden_city)
city_output = Dense(len(np.unique(city_data)), activation='softmax', name='city_output')(merged_hidden_city)

#Class
merged_hidden_class = Dense(2048, activation='relu')(basic_output)
# merged_drop_class = Dropout(0.1)(merged_hidden_class)
class_output = Dense(len(np.unique(class_data)), activation='softmax', name='class_output')(merged_hidden_class)

In [20]:
e256_model = Model(inputs=basic_input, outputs=[city_output, class_output])
# e256_model.compile(optimizer='adam', loss=['categorical_crossentropy', 'categorical_crossentropy'], metrics=['accuracy'])
gamma = 0.5
e256_model.compile(optimizer='adam',
                  loss={'city_output': 'categorical_crossentropy', 
                        'class_output': 'categorical_crossentropy'},
                  loss_weights={'city_output': gamma, 
                                'class_output': 1 - gamma}, 
                  metrics=['accuracy'])

In [21]:
e256_history = e256_model.fit(train_256, {'city_output': train_city, 'class_output': train_class},
                              # batch_size=20,
                              epochs=10,
                              shuffle=True,
                              validation_data=(test_256, {'city_output': test_city, 'class_output': test_class})
                 )

Epoch 1/5
1128/1128 [==============================] - 26s 23ms/step - loss: 10.8059 - city_output_loss: 10.2132 - class_output_loss: 11.3986 - city_output_accuracy: 0.2999 - class_output_accuracy: 0.3300
Epoch 2/5
1128/1128 [==============================] - 25s 23ms/step - loss: 1.7712 - city_output_loss: 1.5705 - class_output_loss: 1.9718 - city_output_accuracy: 0.3009 - class_output_accuracy: 0.3322
Epoch 3/5
1128/1128 [==============================] - 25s 22ms/step - loss: 1.7482 - city_output_loss: 1.5441 - class_output_loss: 1.9523 - city_output_accuracy: 0.3203 - class_output_accuracy: 0.3371
Epoch 4/5
1128/1128 [==============================] - 25s 23ms/step - loss: 1.7387 - city_output_loss: 1.5347 - class_output_loss: 1.9427 - city_output_accuracy: 0.3261 - class_output_accuracy: 0.3471
Epoch 5/5
1128/1128 [==============================] - 26s 23ms/step - loss: 1.7454 - city_output_loss: 1.5399 - class_output_loss: 1.9510 - city_output_accuracy: 0.3201 - class_output_accu

## e512

Get input and initial output layers

In [22]:
basic_input, basic_output = basic_model('main_input')

Attach 2 outputs

In [23]:
#City
merged_hidden_city = Dense(2048, activation='relu')(basic_output)
# merged_drop_city = Dropout(0.1)(merged_hidden_city)
city_output = Dense(len(np.unique(city_data)), activation='softmax', name='city_output')(merged_hidden_city)

#Class
merged_hidden_class = Dense(2048, activation='relu')(basic_output)
# merged_drop_class = Dropout(0.1)(merged_hidden_class)
class_output = Dense(len(np.unique(class_data)), activation='softmax', name='class_output')(merged_hidden_class)

In [24]:
e512_model = Model(inputs=basic_input, outputs=[city_output, class_output])
# e512_model.compile(optimizer='adam', loss=['categorical_crossentropy', 'categorical_crossentropy'], metrics=['accuracy'])
gamma = 0.5
e512_model.compile(optimizer='adam',
                  loss={'city_output': 'categorical_crossentropy', 
                        'class_output': 'categorical_crossentropy'},
                  loss_weights={'city_output': gamma, 
                                'class_output': 1 - gamma}, 
                  metrics=['accuracy'])

In [25]:
e512_history = e512_model.fit(train_512, {'city_output': train_city, 'class_output': train_class},
                              # batch_size=20,
                              epochs=10,
                              shuffle=True,
                              validation_data=(test_512, {'city_output': test_city, 'class_output': test_class})
                 )

Epoch 1/5
1128/1128 [==============================] - 26s 23ms/step - loss: 6.5641 - city_output_loss: 6.7616 - class_output_loss: 6.3666 - city_output_accuracy: 0.3324 - class_output_accuracy: 0.3690
Epoch 2/5
1128/1128 [==============================] - 26s 23ms/step - loss: 1.7037 - city_output_loss: 1.4854 - class_output_loss: 1.9221 - city_output_accuracy: 0.3533 - class_output_accuracy: 0.3820
Epoch 3/5
1128/1128 [==============================] - 26s 23ms/step - loss: 1.6083 - city_output_loss: 1.3770 - class_output_loss: 1.8396 - city_output_accuracy: 0.4100 - class_output_accuracy: 0.4151
Epoch 4/5
1128/1128 [==============================] - 26s 23ms/step - loss: 1.4024 - city_output_loss: 1.1855 - class_output_loss: 1.6192 - city_output_accuracy: 0.5145 - class_output_accuracy: 0.4781
Epoch 5/5
1128/1128 [==============================] - 26s 23ms/step - loss: 1.1268 - city_output_loss: 0.9695 - class_output_loss: 1.2841 - city_output_accuracy: 0.6150 - class_output_accurac

## final

Three inputs
  1. e128
  1. e256
  1. e512

Two outputs
  1. city
  1. class

Create the three basic input models

In [26]:
e128_input, e128_output = basic_model('e128_input')
e256_input, e256_output = basic_model('e256_input')
e512_input, e512_output = basic_model('e512_input')

Concatenating outputs of three models

In [27]:
merged = concatenate([e128_output, e256_output, e512_output])

Defining the outputs

In [28]:
#City
merged_hidden_city = Dense(2048, activation='relu')(merged)
# merged_drop_city = Dropout(0.1)(merged_hidden_city)
city_output = Dense(len(np.unique(city_data)), activation='softmax', name='city_output')(merged_hidden_city)

#Class
merged_hidden_class = Dense(2048, activation='relu')(merged)
# merged_drop_class = Dropout(0.1)(merged_hidden_class)
class_output = Dense(len(np.unique(class_data)), activation='softmax', name='class_output')(merged_hidden_class)

In [29]:
final_model = Model(inputs=[e128_input, e256_input, e512_input], outputs=[city_output, class_output])
# final_model.compile(optimizer='adam', loss=['categorical_crossentropy', 'categorical_crossentropy'], metrics=['accuracy'])
gamma = 0.5
final_model.compile(optimizer='adam',
                  loss={'city_output': 'categorical_crossentropy', 
                        'class_output': 'categorical_crossentropy'},
                  loss_weights={'city_output': gamma, 
                                'class_output': 1 - gamma}, 
                  metrics=['accuracy'])

Training model

In [30]:
final_history = final_model.fit({'e128_input': train_128, 'e256_input': train_256, 'e512_input': train_512}, {'city_output': train_city, 'class_output': train_class},
                                # batch_size=20,
                                epochs=10,
                                shuffle=True,
                                validation_data=({'e128_input': test_128, 'e256_input': test_256, 'e512_input': test_512}, {'city_output': test_city, 'class_output': test_class})
                 )
                

Epoch 1/5
1128/1128 [==============================] - 72s 63ms/step - loss: 14.0932 - city_output_loss: 14.7159 - class_output_loss: 13.4704 - city_output_accuracy: 0.5135 - class_output_accuracy: 0.3899
Epoch 2/5
1128/1128 [==============================] - 71s 63ms/step - loss: 1.2522 - city_output_loss: 0.8827 - class_output_loss: 1.6217 - city_output_accuracy: 0.6633 - class_output_accuracy: 0.4660
Epoch 3/5
1128/1128 [==============================] - 71s 63ms/step - loss: 1.0608 - city_output_loss: 0.8143 - class_output_loss: 1.3074 - city_output_accuracy: 0.6851 - class_output_accuracy: 0.5578
Epoch 4/5
1128/1128 [==============================] - 71s 63ms/step - loss: 0.6673 - city_output_loss: 0.5747 - class_output_loss: 0.7599 - city_output_accuracy: 0.7800 - class_output_accuracy: 0.7388
Epoch 5/5
1128/1128 [==============================] - 71s 63ms/step - loss: 0.4384 - city_output_loss: 0.3849 - class_output_loss: 0.4919 - city_output_accuracy: 0.8600 - class_output_accu

## Evaluate


In [31]:
y_e128_city_pred, y_e128_class_pred = e128_model.predict(test_128)
y_e256_city_pred, y_e256_class_pred = e256_model.predict(test_256)
y_e512_city_pred, y_e512_class_pred = e512_model.predict(test_512)
y_final_city_pred, y_final_class_pred = final_model.predict({'e128_input': test_128, 'e256_input': test_256, 'e512_input': test_512})

177/177 [==============================] - 3s 15ms/step


In [32]:
y_e128_city_pred = np.argmax(y_e128_city_pred, axis=1)
y_e256_city_pred = np.argmax(y_e256_city_pred, axis=1)
y_e512_city_pred = np.argmax(y_e512_city_pred, axis=1)
y_final_city_pred = np.argmax(y_final_city_pred, axis=1)

y_e128_class_pred = np.argmax(y_e128_class_pred, axis=1)
y_e256_class_pred = np.argmax(y_e256_class_pred, axis=1)
y_e512_class_pred = np.argmax(y_e512_class_pred, axis=1)
y_final_class_pred = np.argmax(y_final_class_pred, axis=1)

In [33]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

city_y = np.argmax(test_city, axis=1)
class_y = np.argmax(test_class, axis=1)

e128_city_cm = confusion_matrix(y_e128_city_pred, city_y)
e256_city_cm = confusion_matrix(y_e256_city_pred, city_y)
e512_city_cm = confusion_matrix(y_e512_city_pred, city_y)
final_city_cm = confusion_matrix(y_final_city_pred, city_y)

e128_class_cm = confusion_matrix(y_e128_class_pred, class_y)
e256_class_cm = confusion_matrix(y_e256_class_pred, class_y)
e512_class_cm = confusion_matrix(y_e512_class_pred, class_y)
final_class_cm = confusion_matrix(y_final_class_pred, class_y)

e128_city_f1 = f1_score(y_e128_city_pred, city_y, average='micro')
e256_city_f1 = f1_score(y_e256_city_pred, city_y, average='micro')
e512_city_f1 = f1_score(y_e512_city_pred, city_y, average='micro')
final_city_f1 = f1_score(y_final_city_pred, city_y, average='micro')

e128_class_f1 = f1_score(y_e128_class_pred, class_y, average='micro')
e256_class_f1 = f1_score(y_e256_class_pred, class_y, average='micro')
e512_class_f1 = f1_score(y_e512_class_pred, class_y, average='micro')
final_class_f1 = f1_score(y_final_class_pred, class_y, average='micro')

In [34]:
e128_city_cm

array([[324,  49,  19, 124, 150],
       [ 31, 494, 133, 236,  64],
       [ 51, 109, 417, 111,  66],
       [450, 393, 349, 943, 583],
       [135,  81,  62, 113, 152]])

In [35]:
e256_city_cm

array([[ 113,   29,   49,   48,   91],
       [   3,    0,    3,    1,    5],
       [  32,   11,  237,   28,   32],
       [ 806, 1081,  676, 1441,  865],
       [  37,    5,   15,    9,   22]])

In [36]:
e512_city_cm

array([[579,  50,  54, 141, 248],
       [ 60, 722, 263, 254, 109],
       [ 51, 135, 447, 164,  79],
       [195, 177, 181, 857, 434],
       [106,  42,  35, 111, 145]])

In [37]:
final_city_cm

array([[611,  26, 152, 143, 180],
       [ 41, 923, 193, 110, 195],
       [ 35,  50, 347, 135,  75],
       [153,  46, 162, 896, 210],
       [151,  81, 126, 243, 355]])

In [38]:
e128_class_cm

array([[1161,   21,   12,   48,   57,   12,   24,    1,   37,   93,   51,
          99,    2,  818,   17,   39,   24,   52],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           1,    0,    1,    0,    0,    0,    0],
       [   2,    0,    1,    0,    2,    0,    0,    0,    0,    0,    0,
           1,    0,    2,    0,    0,    0,    0],
       [  47,    1,    2,   51,   39,    5,    8,    1,    0,    2,    6,
          17,    0,  129,    1,    4,    3,    2],
       [  64,    4,    3,   63,   83,    7,   29,    1,    3,   18,   26,
          51,    0,  154,    2,    7,   10,    7],
       [   9,    0,    0,    4,    4,    0,    0,    0,    1,    3,    3,
           2,    0,   10,    0,    1,    0,    0],
       [   5,    0,    0,    4,    6,    2,    4,    0,    0,    3,    4,
           6,    0,   15,    0,    1,    6,    1],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0],


In [39]:
e256_class_cm

array([[1533,   35,   27,  254,  309,   48,  115,    3,   45,  175,  160,
         304,    2, 1522,   27,   77,   82,   83],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0],
       [   2,    0,    0,    1,    2,    0,    1,    0,    1,    2,    0,
           0,    0,    7,    0,    0,    0,    0],
       [   6,    0,    0,    0,    6,    3,    1,    0,    0,    5,    0,
           1,    0,   15,    0,    1,    0,    2],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0],


In [40]:
e512_class_cm

array([[ 998,   16,    8,   42,   41,   11,   19,    0,   24,   69,   48,
          83,    1,  534,    7,   27,   16,   27],
       [   1,    1,    0,    0,    0,    0,    1,    0,    0,    0,    1,
           0,    0,    4,    0,    0,    0,    0],
       [   1,    0,    0,    0,    0,    0,    1,    0,    0,    0,    0,
           0,    0,    6,    0,    0,    0,    1],
       [  11,    0,    0,   10,   11,    1,    2,    0,    1,    0,    2,
           4,    0,   30,    2,    1,    4,    2],
       [  28,    1,    1,   27,   75,    4,   26,    0,    2,   21,   21,
          40,    0,   85,    1,    3,    4,    6],
       [   1,    0,    1,    0,    2,    0,    3,    0,    0,    0,    2,
           1,    0,    2,    1,    0,    1,    0],
       [   3,    0,    1,    6,   12,    0,    5,    0,    0,    2,    1,
           6,    0,   12,    0,    1,    4,    1],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    1,    0,    1,    0,    0],


In [41]:
final_class_cm

array([[872,  18,   9,  19,  24,   4,   5,   1,  19,  80,  23,  48,   0,
        458,   9,  20,  12,  25],
       [  2,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0,   1,   0,
          1,   0,   0,   0,   0],
       [  4,   0,   0,   1,   2,   0,   0,   0,   0,   1,   0,   3,   0,
          0,   0,   0,   0,   0],
       [ 36,   0,   3,  45,  39,   5,   4,   2,   1,   2,   8,  22,   0,
        121,   0,   3,   7,   0],
       [ 47,   2,   2,  36,  45,   7,  13,   0,   3,   8,  13,  31,   0,
        119,   0,   6,  13,   2],
       [  3,   0,   1,   3,   0,   0,   3,   0,   0,   2,   3,   4,   0,
          6,   0,   1,   0,   1],
       [ 33,   0,   3,  14,  59,   8,  22,   1,   1,   3,  20,  40,   0,
         72,   0,   4,  10,   2],
       [  0,   0,   0,   0,   1,   0,   0,   0,   1,   0,   1,   0,   0,
          2,   0,   0,   0,   0],
       [  3,   0,   0,   0,   0,   0,   0,   0,   0,   1,   0,   1,   0,
          6,   0,   0,   1,   1],
       [ 59,   1,   0,   1,  10,   0,